<a href="https://colab.research.google.com/github/dutreras369/bsale-api-flask/blob/master/Bsale_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Dependencias
Primero, asegúrate de instalar las dependencias necesarias en tu Colab:

In [ ]:
!pip install flask flask-ngrok pandas requests



2. Configuración Inicial del Flask App
Este es el código base para restaurar tu servidor Flask:



In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
import requests

app = Flask(__name__)
CORS(app, resources={r"/*": {"origins": "*"}})  # Permite todas las orígenes

# Configuración de la API de Bsale
BSALE_BASE_URL = "https://api.bsale.io/v1/"
BSALE_API_KEY = "9da662145775dea91ce49c52e00ea760190c46c6"  # Reemplaza con tu API Key de Bsale
HEADERS = {"access_token": BSALE_API_KEY}


*italicized text*

In [ ]:
Obtener Productos desde Bsale

In [ ]:
def fetch_products(limit=10):
    """Obtener un número limitado de productos desde Bsale."""
    products = []
    url = f"{BSALE_BASE_URL}products.json?limit={limit}"

    response = requests.get(url, headers=HEADERS).json()
    if "error" in response:
        raise Exception(f"API Error: {response['error']}")

    for item in response.get("items", []):
        product = {
            "name": item.get("name"),
            "id": item.get("id"),
            "items": []  # Variantes del producto
        }

        # Obtener variantes del producto
        variants_href = item.get("variants", {}).get("href")
        if variants_href:
            product["items"] = fetch_variants(variants_href)

        products.append(product)

    return products




```
# This is formatted as code
```



In [ ]:
Obtener Variantes

In [ ]:
def fetch_variants(variants_url):
    """Obtener variaciones de producto desde Bsale."""
    response = requests.get(variants_url, headers=HEADERS).json()
    if "error" in response:
        raise Exception(f"Variants API Error: {response['error']}")

    return [
        {
            "id": variant.get("id"),
            "barcode": variant.get("barCode"),
            "sku": variant.get("code"),
            "description": variant.get("description"),
            "costs": fetch_variant_cost(variant.get("costs", {}).get("href")),
        }
        for variant in response.get("items", [])
    ]



Obtener Costos

In [ ]:
def fetch_variant_cost(costs_url):
    """Obtener el costo promedio de una variación."""
    if not costs_url:
        return 0
    response = requests.get(costs_url, headers=HEADERS).json()
    return response.get("averageCost", 0)


Obtener Stock

In [ ]:
def fetch_stocks(limit=10):
    """Obtener un número limitado de stocks desde Bsale."""
    stocks = []
    url = f"{BSALE_BASE_URL}stocks.json?limit={limit}"

    response = requests.get(url, headers=HEADERS).json()
    if "error" in response:
        raise Exception(f"Stock API Error: {response['error']}")

    return response.get("items", [])


Consolidar


In [ ]:
def consolidate_data(products, stocks):
    consolidated = []

    for product in products:
        consolidated_product = {
            "name": product["name"],
            "id": product["id"],
            "items": []
        }

        for item in product["items"]:
            variant_id = item["id"]

            # Buscar stock asociado
            stock_match = next(
                (stock for stock in stocks if stock.get("variant", {}).get("id") == str(variant_id)),
                None
            )

            if stock_match:
                item["stock"] = {
                    "id": stock_match.get("id"),
                    "quantity": stock_match.get("quantity", 0),
                    "quantityReserved": stock_match.get("quantityReserved", 0),
                    "quantityAvailable": stock_match.get("quantityAvailable", 0)
                }

            consolidated_product["items"].append(item)

        if consolidated_product["items"]:
            consolidated.append(consolidated_product)

    return consolidated


Controller

In [ ]:
@app.route('/sync', methods=['GET'])
def sync_data():
    try:
        products = fetch_products(limit=10)
        stocks = fetch_stocks(limit=10)
        consolidated_data = consolidate_data(products, stocks)

        return jsonify({"products": consolidated_data}), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500


Server


In [ ]:
# Exponer el puerto 5000 usando ngrok
public_url = ngrok.connect(5000)
print(f"URL pública de ngrok: {public_url}")

if __name__ == '__main__':
    app.run(port=5000)

